# 📈 Sales Analytics Dashboard
### Retail E-Commerce | 3-Year Performance Analysis

**Author:** Ankit Jinkwan | **Portfolio:** [ankitjhinkwan.github.io/portfolio](https://ankitjhinkwan.github.io/portfolio/)

---

### 🎯 Goal
Analyse 3 years of retail sales data to uncover trends, top products, regional performance and customer behaviour.

### 📂 Dataset
- 5,000 orders | 6 categories | 15 cities | 2022–2024

### 🔧 Steps
1. Import Libraries
2. Load & Explore
3. Revenue Trends
4. Category Analysis
5. Regional Performance
6. Channel & Payment
7. Product Performance
8. KPI Summary

## Step 1 — Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 5)
print('✅ Libraries loaded!')

## Step 2 — Load & Explore Data

In [ ]:
df = pd.read_csv('../data/sales_data.csv', parse_dates=['Date'])
df['Month'] = df['Date'].dt.to_period('M')
df['DayOfWeek'] = df['Date'].dt.day_name()
print(f'Shape: {df.shape}')
print(f'Date Range: {df["Date"].min().date()} to {df["Date"].max().date()}')
print(f'Total Revenue: Rs.{df["Revenue"].sum():,.0f}')
df.head()

In [ ]:
# KPI Summary
print(f'Total Revenue:   Rs.{df["Revenue"].sum():,.0f}')
print(f'Total Profit:    Rs.{df["Profit"].sum():,.0f}')
print(f'Total Orders:    {len(df):,}')
print(f'Avg Order Value: Rs.{df["Revenue"].mean():,.0f}')
print(f'Profit Margin:   {df["Profit"].sum()/df["Revenue"].sum()*100:.1f}%')
print(f'Return Rate:     {df["Returned"].mean()*100:.1f}%')
print(f'Avg Rating:      {df["Rating"].mean():.2f}')

## Step 3 — Revenue Trends

In [ ]:
monthly = df.groupby('Month').agg(Revenue=('Revenue','sum'), Profit=('Profit','sum'), Orders=('OrderID','count')).reset_index()
monthly['Month_str'] = monthly['Month'].astype(str)

fig, axes = plt.subplots(2, 1, figsize=(14, 8))
ax1 = axes[0]
bars = ax1.bar(range(len(monthly)), monthly['Revenue'], color='#13d0d0', alpha=0.8, label='Revenue')
ax2 = ax1.twinx()
ax2.plot(range(len(monthly)), monthly['Profit'], color='#ff6b6b', lw=2.5, marker='o', ms=3, label='Profit')
ax1.set_xticks(range(len(monthly))); ax1.set_xticklabels(monthly['Month_str'], rotation=45, ha='right', fontsize=8)
ax1.set_ylabel('Revenue'); ax2.set_ylabel('Profit')
ax1.set_title('Monthly Revenue & Profit Trend', fontsize=13, fontweight='bold')
lines1,labels1 = ax1.get_legend_handles_labels(); lines2,labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1+lines2, labels1+labels2)

axes[1].plot(range(len(monthly)), monthly['Orders'], color='#ffd93d', lw=2, marker='s', ms=4)
axes[1].fill_between(range(len(monthly)), monthly['Orders'], alpha=0.2, color='#ffd93d')
axes[1].set_xticks(range(len(monthly))); axes[1].set_xticklabels(monthly['Month_str'], rotation=45, ha='right', fontsize=8)
axes[1].set_ylabel('Orders'); axes[1].set_title('Monthly Order Volume', fontsize=13, fontweight='bold')
plt.tight_layout(); plt.savefig('../revenue_trend.png', dpi=150, bbox_inches='tight'); plt.show()

## Step 4 — Category Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
colors = ['#13d0d0','#ff6b6b','#ffd93d','#10b981','#a78bfa','#fb923c']
cat_rev = df.groupby('Category')['Revenue'].sum().sort_values(ascending=False)
axes[0].bar(cat_rev.index, cat_rev.values, color=colors, edgecolor='white')
axes[0].set_title('Revenue by Category', fontweight='bold'); axes[0].tick_params(axis='x', rotation=15)
cat_margin = (df.groupby('Category')['Profit'].sum()/df.groupby('Category')['Revenue'].sum()*100).sort_values(ascending=False)
axes[1].bar(cat_margin.index, cat_margin.values, color=colors, edgecolor='white')
axes[1].set_title('Profit Margin by Category', fontweight='bold'); axes[1].set_ylabel('Margin (%)'); axes[1].tick_params(axis='x', rotation=15)
cat_orders = df.groupby('Category')['OrderID'].count()
axes[2].pie(cat_orders.values, labels=cat_orders.index, colors=colors, autopct='%1.1f%%', startangle=90)
axes[2].set_title('Order Share', fontweight='bold')
plt.tight_layout(); plt.savefig('../category_analysis.png', dpi=150, bbox_inches='tight'); plt.show()

## Step 5 — Regional Performance

In [ ]:
regional = df.groupby('Region').agg(Revenue=('Revenue','sum'), Profit=('Profit','sum'), Orders=('OrderID','count')).round(2)
regional['Margin'] = (regional['Profit']/regional['Revenue']*100).round(1)
print(regional.to_string())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
rs = regional.sort_values('Revenue', ascending=True)
axes[0].barh(rs.index, rs['Revenue'], color='#13d0d0', edgecolor='white')
axes[0].set_title('Revenue by Region', fontweight='bold')
axes[1].bar(regional.index, regional['Margin'], color='#ff6b6b', edgecolor='white')
axes[1].set_title('Profit Margin by Region', fontweight='bold'); axes[1].set_ylabel('Margin (%)')
plt.tight_layout(); plt.savefig('../regional_analysis.png', dpi=150, bbox_inches='tight'); plt.show()

## Step 6 — Channel & Payment

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = ['#13d0d0','#ff6b6b','#ffd93d','#10b981']
channel = df.groupby('Channel')['Revenue'].sum().sort_values(ascending=False)
axes[0].bar(channel.index, channel.values, color=colors, edgecolor='white')
axes[0].set_title('Revenue by Channel', fontweight='bold')
for i,v in enumerate(channel.values): axes[0].text(i, v+1000, f'Rs.{v/1000:.0f}K', ha='center', fontsize=9, fontweight='bold')
payment = df['PaymentMethod'].value_counts()
axes[1].pie(payment.values, labels=payment.index, colors=['#13d0d0','#ff6b6b','#ffd93d','#10b981','#a78bfa'], autopct='%1.1f%%')
axes[1].set_title('Payment Methods', fontweight='bold')
plt.tight_layout(); plt.savefig('../channel_payment.png', dpi=150, bbox_inches='tight'); plt.show()

## Step 7 — Product Performance

In [ ]:
top10 = df.groupby('Product').agg(Revenue=('Revenue','sum'), Orders=('OrderID','count'), AvgRating=('Rating','mean')).nlargest(10,'Revenue').round(2)
print('Top 10 Products:')
print(top10.to_string())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
ts = top10.sort_values('Revenue', ascending=True)
axes[0].barh(ts.index, ts['Revenue'], color=plt.cm.cool(np.linspace(0.3,1,10)), edgecolor='white')
axes[0].set_title('Top 10 Products by Revenue', fontweight='bold')
axes[1].scatter(top10['Orders'], top10['Revenue'], s=top10['AvgRating']*50, c=range(len(top10)), cmap='cool', alpha=0.8)
for idx,row in top10.iterrows(): axes[1].annotate(idx,(row['Orders'],row['Revenue']),fontsize=7,ha='center')
axes[1].set_xlabel('Orders'); axes[1].set_ylabel('Revenue'); axes[1].set_title('Orders vs Revenue', fontweight='bold')
plt.tight_layout(); plt.savefig('../product_analysis.png', dpi=150, bbox_inches='tight'); plt.show()

## Step 8 — KPI Summary

In [ ]:
print('='*50)
print('     SALES ANALYTICS KPI SUMMARY')
print('='*50)
print(f'  Total Revenue:   Rs.{df["Revenue"].sum():>12,.0f}')
print(f'  Total Profit:    Rs.{df["Profit"].sum():>12,.0f}')
print(f'  Profit Margin:   {df["Profit"].sum()/df["Revenue"].sum()*100:>11.1f}%')
print(f'  Total Orders:    {len(df):>12,}')
print(f'  Avg Order Value: Rs.{df["Revenue"].mean():>12,.0f}')
print(f'  Return Rate:     {df["Returned"].mean()*100:>11.1f}%')
print(f'  Avg Rating:      {df["Rating"].mean():>12.2f}')
print(f'  Best Category:   {df.groupby("Category")["Revenue"].sum().idxmax():>12}')
print(f'  Best Region:     {df.groupby("Region")["Revenue"].sum().idxmax():>12}')
print(f'  Best Channel:    {df.groupby("Channel")["Revenue"].sum().idxmax():>12}')
print('='*50)
print('\nRun the dashboard: streamlit run app/app.py')

## ✅ Key Insights

| Metric | Value |
|--------|-------|
| Total Revenue | ₹19.5L+ |
| Profit Margin | ~27% |
| Best Category | Electronics |
| Top Channel | Website |
| Peak Season | Oct–Dec |

### 🔑 Findings
- **Electronics** drives most revenue but **Books** has highest margin
- **Festival season** accounts for ~35% of annual revenue
- **UPI** is most popular payment method (30%+)
- **Mobile App** growing fastest year-over-year
- **North & West** regions dominate revenue

---
*Built by Ankit Jinkwan | [Portfolio](https://ankitjhinkwan.github.io/portfolio/)*